In [1]:
# relevant imports
import numpy as np
from scipy import io
import matplotlib.pyplot as plt
from matplotlib import rcParams
import rsatoolbox
import rsatoolbox.data as rsd # abbreviation to deal with dataset
import rsatoolbox.rdm as rsr
import os
import seaborn as sns
import sklearn as sk
import math
import pandas as pd
import pickle
import copy
import nibabel as nib
import re
from statsmodels.stats.multitest import multipletests

In [2]:
# change the working directory to be the timecourse data
#os.chdir('/mnt/Data1/linjdata1/vswmda/scan_data/rsa/full_GLM_mgs_0.05_50/')
os.chdir('/gpfs/scratch/linjjiang/scan_data/rsa/full_GLM_mgs_topN/')
#os.chdir('/gpfs/scratch/linjjiang/scan_data/rsa/full_GLM_atlas_roi/')

In [3]:
with open('crossnobis_dual_bootstrap_10000_exp1.pkg','rb') as f:
    df1 = pickle.load(f)   

In [4]:
roi_all = df1['roi'].unique()
session_all = df1['session'].unique()
epoch_all = ['stimulus','delay']

In [5]:
data = []

# session = 0
# epoch = 'delay'
# roi = 'v1'

for roi in roi_all:
    for epoch in epoch_all:
        for session in session_all:
            #print(roi,epoch,session)
            df = df1[(df1['roi']==roi) & (df1['session']==session) & (df1['epoch']==epoch)]
            results = df.iloc[0,0] 

            models = results.models
            models_name = [m.name for m in models]

            # Extract the test results #test_type='bootstrap', test_type='bootstrap', test_type='bootstrap'
            pairwise_results = results.test_all()[0]
            test_against_zero = results.test_all()[1]
            test_against_noise_ceiling = results.test_all()[2]
            
#             pairwise_results = results.test_pairwise(test_type='bootstrap')
#             test_against_zero = results.test_zero(test_type='bootstrap').mean(axis=(1, 2, 3))
#             test_against_noise_ceiling = results.test_noise(test_type='t-test')
            
            eval_mean = results.get_means()
            eval_sem = results.get_sem()
            noise_ceil = results.get_noise_ceil()

            # Store pairwise test results
            for i, model1 in enumerate(models_name):
                for j, model2 in enumerate(models_name):
                    data.append({
                        'model': f'{model1} vs {model2}',                
                        'session': session,
                        'epoch': epoch,
                        'roi': roi,
                        'comparison': 'pairwise',
                        'mean': [eval_mean[i],eval_mean[j]],
                        'mean_diff':eval_mean[i]-eval_mean[j],
                        'p_value': pairwise_results[i, j],
                        'sem': [eval_sem[i],eval_sem[j]],
            #            'noise_ceil': noise_ceil
                    })

            # Store test against zero results
            for i, model in enumerate(models_name):
                data.append({
                    'model': model,
                    'session': session,
                    'epoch': epoch,
                    'roi': roi,            
                    'comparison': 'against_zero',
                        'mean': eval_mean[i],
                    'p_value': test_against_zero[i],
                        'sem': eval_sem[i],
            #            'noise_ceil': noise_ceil
                })

            # Store test against noise ceiling results
            for i, model in enumerate(models_name):
                data.append({
                    'model': model,
                    'session': session,
                    'epoch': epoch,
                    'roi': roi,

                    'comparison': 'against_noise_ceiling',

                        'mean': eval_mean[i],
                    'p_value': test_against_noise_ceiling[i],
                        'sem': eval_sem[i],
            #            'noise_ceil': noise_ceil
                })

/gpfs/home/linjjiang/.local/lib/python3.11/site-packages/rsatoolbox/inference/result.py:222: RuntimeWarning: Mean of empty slice
  perf = np.nanmean(perf, axis=-1)


In [6]:
# Convert the list of dictionaries to a DataFrame
df_results = pd.DataFrame(data)

In [7]:
roi_labels = {'area4': 'M1', 'v1': 'V1', 'v2': 'V2', 
              
              'ips0': 'IPS0', 'ips1': 'IPS1', 'ips2': 'IPS2', 
              'ips3': 'IPS3', 'ips4': 'IPS4', 'ips5': 'IPS5', 'spl1': 'SPL1',
              
          'ips': 'IPS', 'fef': 'FEF', 'sfg': 'SFG', 
          'mfg': 'MFG', 'ifg': 'IFG'}

In [8]:
df_results['roi'] = df_results['roi'].map(roi_labels)

In [9]:
df1['roi'] = df1['roi'].map(roi_labels)

In [10]:
import warnings
def multi_corr(result_df):
    warnings.filterwarnings('ignore')
    # Correct for multiple comparisons
    p_values = result_df['p_value'].values.flatten()
    corrected_p_values = multipletests(p_values, method='fdr_bh')[1]
    result_df['p_value_corr'] = corrected_p_values.reshape(-1, 1)
    
    return result_df

In [11]:
print(df_results)

             model  session     epoch   roi             comparison  \
0         LR vs LR        0  stimulus    M1               pairwise   
1         LR vs WT        0  stimulus    M1               pairwise   
2        LR vs Ecc        0  stimulus    M1               pairwise   
3      LR vs LR-WT        0  stimulus    M1               pairwise   
4     LR vs Ecc-LR        0  stimulus    M1               pairwise   
...            ...      ...       ...   ...                    ...   
3775           Ecc        1     delay  SPL1  against_noise_ceiling   
3776         LR-WT        1     delay  SPL1  against_noise_ceiling   
3777        Ecc-LR        1     delay  SPL1  against_noise_ceiling   
3778        Ecc-WT        1     delay  SPL1  against_noise_ceiling   
3779           All        1     delay  SPL1  against_noise_ceiling   

                                                mean  mean_diff   p_value  \
0     [-0.007764248858685116, -0.007764248858685116]   0.000000  1.000000   
1    

In [12]:
print(df_results[df_results['comparison']=='pairwise']['p_value'])

0       1.000000
1       0.933693
2       0.710128
3       0.716458
4       0.627245
          ...   
3761    0.503370
3762    0.250709
3763    0.174926
3764    0.657917
3765    1.000000
Name: p_value, Length: 2940, dtype: float64


In [13]:
print(df_results[df_results['comparison']=='against_zero']['p_value'])

49      0.564977
50      0.572206
51      0.386565
52      0.344719
53      0.343213
          ...   
3768    0.041166
3769    0.024812
3770    0.190430
3771    0.005236
3772    0.010478
Name: p_value, Length: 420, dtype: float64


In [14]:
test_against_zero = results.test_zero(test_type='bootstrap')
print(test_against_zero.shape)

(7, 4, 2, 3)


# Against zero

In [164]:
df_temp = pd.DataFrame()

In [165]:
roi_order = ['V1', 'V2']
df = df_results[(df_results['comparison'].isin(['against_zero']))
          & (df_results['roi'].isin(roi_order))
                & (df_results['model'].isin(['LR']))] #            & (df_results['p_value']<=0.05)
df = multi_corr(df)
print(df)

df_temp = pd.concat((df_temp,df),axis=0)

    model  session     epoch roi    comparison      mean  mean_diff   p_value  \
301    LR        0  stimulus  V1  against_zero  0.016844        NaN  0.338135   
364    LR        1  stimulus  V1  against_zero  0.079157        NaN  0.052310   
427    LR        0     delay  V1  against_zero -0.001971        NaN  0.508348   
490    LR        1     delay  V1  against_zero -0.058307        NaN  0.726613   
553    LR        0  stimulus  V2  against_zero  0.019642        NaN  0.283514   
616    LR        1  stimulus  V2  against_zero   0.07166        NaN  0.021404   
679    LR        0     delay  V2  against_zero  0.006523        NaN  0.472347   
742    LR        1     delay  V2  against_zero  0.050776        NaN  0.268313   

          sem  p_value_corr  
301  0.038023      0.541016  
364  0.037842      0.209241  
427    0.0896      0.580969  
490  0.088681      0.726613  
553  0.032072      0.541016  
616  0.024456      0.171230  
679  0.089449      0.580969  
742  0.075212      0.541016  


In [166]:
df = df_results[(df_results['comparison'].isin(['against_zero']))
          & (df_results['roi'].isin(roi_order))
                & (df_results['model'].isin(['WT']))] #            & (df_results['p_value']<=0.05)
df = multi_corr(df)
print(df)

df_temp = pd.concat((df_temp,df),axis=0)

    model  session     epoch roi    comparison      mean  mean_diff   p_value  \
302    WT        0  stimulus  V1  against_zero  0.178516        NaN  0.004071   
365    WT        1  stimulus  V1  against_zero  0.230689        NaN  0.003632   
428    WT        0     delay  V1  against_zero  0.096234        NaN  0.124460   
491    WT        1     delay  V1  against_zero  0.038062        NaN  0.283504   
554    WT        0  stimulus  V2  against_zero  0.165382        NaN  0.006333   
617    WT        1  stimulus  V2  against_zero  0.223397        NaN  0.001500   
680    WT        0     delay  V2  against_zero  0.044339        NaN  0.268054   
743    WT        1     delay  V2  against_zero  0.106963        NaN  0.095782   

          sem  p_value_corr  
302  0.042069      0.010857  
365  0.045739      0.010857  
428  0.073778      0.165947  
491  0.061088      0.283504  
554  0.043555      0.012667  
617  0.034715      0.010857  
680   0.06678      0.283504  
743  0.068141      0.153251  


In [167]:
df = df_results[(df_results['comparison'].isin(['against_zero']))
          & (df_results['roi'].isin(roi_order))
                & (df_results['model'].isin(['Ecc']))] #            & (df_results['p_value']<=0.05)
df = multi_corr(df)
print(df)

df_temp = pd.concat((df_temp,df),axis=0)

    model  session     epoch roi    comparison      mean  mean_diff   p_value  \
303   Ecc        0  stimulus  V1  against_zero  0.204376        NaN  0.002473   
366   Ecc        1  stimulus  V1  against_zero  0.217463        NaN  0.003609   
429   Ecc        0     delay  V1  against_zero -0.019025        NaN  0.615660   
492   Ecc        1     delay  V1  against_zero -0.005997        NaN  0.528913   
555   Ecc        0  stimulus  V2  against_zero  0.233801        NaN  0.000496   
618   Ecc        1  stimulus  V2  against_zero  0.246806        NaN  0.003839   
681   Ecc        0     delay  V2  against_zero -0.084086        NaN  0.847690   
744   Ecc        1     delay  V2  against_zero  0.025533        NaN  0.374320   

          sem  p_value_corr  
303  0.042707      0.007679  
366  0.043039      0.007679  
429  0.061264      0.703612  
492  0.077685      0.703612  
555  0.033975      0.003966  
618  0.049709      0.007679  
681  0.073536      0.847690  
744   0.07437      0.598912  


In [168]:
roi_order = ['IPS0', 'IPS1', 'IPS2', 'SPL1']
df = df_results[(df_results['comparison'].isin(['against_zero']))
          & (df_results['roi'].isin(roi_order))
                & (df_results['model'].isin(['LR']))] #            & (df_results['p_value']<=0.05)
df = multi_corr(df)
print(df)

df_temp = pd.concat((df_temp,df),axis=0)

     model  session     epoch   roi    comparison      mean  mean_diff  \
2065    LR        0  stimulus  IPS0  against_zero   0.00713        NaN   
2128    LR        1  stimulus  IPS0  against_zero  0.057072        NaN   
2191    LR        0     delay  IPS0  against_zero -0.004372        NaN   
2254    LR        1     delay  IPS0  against_zero -0.023772        NaN   
2317    LR        0  stimulus  IPS1  against_zero  0.008019        NaN   
2380    LR        1  stimulus  IPS1  against_zero -0.001236        NaN   
2443    LR        0     delay  IPS1  against_zero -0.048156        NaN   
2506    LR        1     delay  IPS1  against_zero  0.001384        NaN   
2569    LR        0  stimulus  IPS2  against_zero -0.070683        NaN   
2632    LR        1  stimulus  IPS2  against_zero -0.036883        NaN   
2695    LR        0     delay  IPS2  against_zero  0.024605        NaN   
2758    LR        1     delay  IPS2  against_zero -0.003847        NaN   
3577    LR        0  stimulus  SPL1  a

In [169]:
df = df_results[(df_results['comparison'].isin(['against_zero']))
          & (df_results['roi'].isin(roi_order))
                & (df_results['model'].isin(['WT']))] #            & (df_results['p_value']<=0.05)
df = multi_corr(df)
print(df)

df_temp = pd.concat((df_temp,df),axis=0)

     model  session     epoch   roi    comparison      mean  mean_diff  \
2066    WT        0  stimulus  IPS0  against_zero  0.022315        NaN   
2129    WT        1  stimulus  IPS0  against_zero  0.090254        NaN   
2192    WT        0     delay  IPS0  against_zero  0.128713        NaN   
2255    WT        1     delay  IPS0  against_zero  0.115978        NaN   
2318    WT        0  stimulus  IPS1  against_zero  0.028637        NaN   
2381    WT        1  stimulus  IPS1  against_zero -0.026878        NaN   
2444    WT        0     delay  IPS1  against_zero  0.089264        NaN   
2507    WT        1     delay  IPS1  against_zero  0.119114        NaN   
2570    WT        0  stimulus  IPS2  against_zero  0.008222        NaN   
2633    WT        1  stimulus  IPS2  against_zero  -0.03729        NaN   
2696    WT        0     delay  IPS2  against_zero  0.101306        NaN   
2759    WT        1     delay  IPS2  against_zero  0.109638        NaN   
3578    WT        0  stimulus  SPL1  a

In [170]:
df = df_results[(df_results['comparison'].isin(['against_zero']))
          & (df_results['roi'].isin(roi_order))
                & (df_results['model'].isin(['Ecc']))] #            & (df_results['p_value']<=0.05)
df = multi_corr(df)
print(df)

df_temp = pd.concat((df_temp,df),axis=0)

     model  session     epoch   roi    comparison      mean  mean_diff  \
2067   Ecc        0  stimulus  IPS0  against_zero  0.136649        NaN   
2130   Ecc        1  stimulus  IPS0  against_zero  0.115812        NaN   
2193   Ecc        0     delay  IPS0  against_zero  0.050241        NaN   
2256   Ecc        1     delay  IPS0  against_zero  0.142386        NaN   
2319   Ecc        0  stimulus  IPS1  against_zero  0.085936        NaN   
2382   Ecc        1  stimulus  IPS1  against_zero -0.009342        NaN   
2445   Ecc        0     delay  IPS1  against_zero  0.012808        NaN   
2508   Ecc        1     delay  IPS1  against_zero  0.108449        NaN   
2571   Ecc        0  stimulus  IPS2  against_zero -0.036932        NaN   
2634   Ecc        1  stimulus  IPS2  against_zero -0.005898        NaN   
2697   Ecc        0     delay  IPS2  against_zero  0.058828        NaN   
2760   Ecc        1     delay  IPS2  against_zero   0.12216        NaN   
3579   Ecc        0  stimulus  SPL1  a

In [171]:
roi_order = ['FEF', 'SFG', 'MFG', 'IFG']
df = df_results[(df_results['comparison'].isin(['against_zero']))
          & (df_results['roi'].isin(roi_order))
                & (df_results['model'].isin(['LR']))] #            & (df_results['p_value']<=0.05)
df = multi_corr(df)
print(df)

df_temp = pd.concat((df_temp,df),axis=0)

     model  session     epoch  roi    comparison      mean  mean_diff  \
1057    LR        0  stimulus  FEF  against_zero -0.046209        NaN   
1120    LR        1  stimulus  FEF  against_zero  0.036424        NaN   
1183    LR        0     delay  FEF  against_zero -0.026913        NaN   
1246    LR        1     delay  FEF  against_zero  -0.03673        NaN   
1309    LR        0  stimulus  SFG  against_zero  0.024377        NaN   
1372    LR        1  stimulus  SFG  against_zero -0.030704        NaN   
1435    LR        0     delay  SFG  against_zero -0.039715        NaN   
1498    LR        1     delay  SFG  against_zero -0.054384        NaN   
1561    LR        0  stimulus  MFG  against_zero -0.006489        NaN   
1624    LR        1  stimulus  MFG  against_zero  0.009767        NaN   
1687    LR        0     delay  MFG  against_zero -0.028471        NaN   
1750    LR        1     delay  MFG  against_zero -0.030926        NaN   
1813    LR        0  stimulus  IFG  against_zero -0

In [172]:
df = df_results[(df_results['comparison'].isin(['against_zero']))
          & (df_results['roi'].isin(roi_order))
                & (df_results['model'].isin(['WT']))] #            & (df_results['p_value']<=0.05)
df = multi_corr(df)
print(df)

df_temp = pd.concat((df_temp,df),axis=0)

     model  session     epoch  roi    comparison      mean  mean_diff  \
1058    WT        0  stimulus  FEF  against_zero -0.037983        NaN   
1121    WT        1  stimulus  FEF  against_zero  0.039796        NaN   
1184    WT        0     delay  FEF  against_zero  0.051181        NaN   
1247    WT        1     delay  FEF  against_zero  0.073475        NaN   
1310    WT        0  stimulus  SFG  against_zero -0.012036        NaN   
1373    WT        1  stimulus  SFG  against_zero -0.034879        NaN   
1436    WT        0     delay  SFG  against_zero -0.009186        NaN   
1499    WT        1     delay  SFG  against_zero -0.005205        NaN   
1562    WT        0  stimulus  MFG  against_zero  -0.03584        NaN   
1625    WT        1  stimulus  MFG  against_zero -0.008131        NaN   
1688    WT        0     delay  MFG  against_zero -0.051181        NaN   
1751    WT        1     delay  MFG  against_zero  0.032961        NaN   
1814    WT        0  stimulus  IFG  against_zero -0

In [173]:
df = df_results[(df_results['comparison'].isin(['against_zero']))
          & (df_results['roi'].isin(roi_order))
                & (df_results['model'].isin(['Ecc']))] #            & (df_results['p_value']<=0.05)
df = multi_corr(df)
print(df)

df_temp = pd.concat((df_temp,df),axis=0)

     model  session     epoch  roi    comparison      mean  mean_diff  \
1059   Ecc        0  stimulus  FEF  against_zero  0.045026        NaN   
1122   Ecc        1  stimulus  FEF  against_zero  0.035575        NaN   
1185   Ecc        0     delay  FEF  against_zero  0.037126        NaN   
1248   Ecc        1     delay  FEF  against_zero -0.019595        NaN   
1311   Ecc        0  stimulus  SFG  against_zero -0.004231        NaN   
1374   Ecc        1  stimulus  SFG  against_zero  0.004751        NaN   
1437   Ecc        0     delay  SFG  against_zero -0.004854        NaN   
1500   Ecc        1     delay  SFG  against_zero  0.036051        NaN   
1563   Ecc        0  stimulus  MFG  against_zero  0.020192        NaN   
1626   Ecc        1  stimulus  MFG  against_zero  0.076685        NaN   
1689   Ecc        0     delay  MFG  against_zero -0.014132        NaN   
1752   Ecc        1     delay  MFG  against_zero  0.058312        NaN   
1815   Ecc        0  stimulus  IFG  against_zero  0

In [174]:
print(df_temp)

     model  session     epoch  roi    comparison      mean  mean_diff  \
301     LR        0  stimulus   V1  against_zero  0.016844        NaN   
364     LR        1  stimulus   V1  against_zero  0.079157        NaN   
427     LR        0     delay   V1  against_zero -0.001971        NaN   
490     LR        1     delay   V1  against_zero -0.058307        NaN   
553     LR        0  stimulus   V2  against_zero  0.019642        NaN   
...    ...      ...       ...  ...           ...       ...        ...   
1752   Ecc        1     delay  MFG  against_zero  0.058312        NaN   
1815   Ecc        0  stimulus  IFG  against_zero  0.039467        NaN   
1878   Ecc        1  stimulus  IFG  against_zero -0.006256        NaN   
1941   Ecc        0     delay  IFG  against_zero  -0.03786        NaN   
2004   Ecc        1     delay  IFG  against_zero  0.119973        NaN   

       p_value       sem  p_value_corr  
301   0.338135  0.038023      0.541016  
364   0.052310  0.037842      0.209241  


In [175]:
df_test = df_temp[df_temp['p_value']<0.05]
print(df_test)

     model  session     epoch   roi    comparison      mean  mean_diff  \
616     LR        1  stimulus    V2  against_zero   0.07166        NaN   
302     WT        0  stimulus    V1  against_zero  0.178516        NaN   
365     WT        1  stimulus    V1  against_zero  0.230689        NaN   
554     WT        0  stimulus    V2  against_zero  0.165382        NaN   
617     WT        1  stimulus    V2  against_zero  0.223397        NaN   
303    Ecc        0  stimulus    V1  against_zero  0.204376        NaN   
366    Ecc        1  stimulus    V1  against_zero  0.217463        NaN   
555    Ecc        0  stimulus    V2  against_zero  0.233801        NaN   
618    Ecc        1  stimulus    V2  against_zero  0.246806        NaN   
2255    WT        1     delay  IPS0  against_zero  0.115978        NaN   
2696    WT        0     delay  IPS2  against_zero  0.101306        NaN   
3767    WT        1     delay  SPL1  against_zero  0.141067        NaN   
2067   Ecc        0  stimulus  IPS0  a

In [42]:
df_test.to_csv('tbl_bootstrap_RDM_accuracy_against_zero.csv')

# pairwise across task epochs

In [15]:
from rsatoolbox.util.inference_util import extract_variances
from scipy import stats
def calc_model_var(evaluations,n_rdm,n_pattern):
    eval_ok = (~np.isnan(evaluations[:, 0, 0, 0, 0]) & ~np.isnan(evaluations[:, 1, 0, 0, 0]))
    
    # we essentially project from the two points for 1 repetition and
    # for n_cv repetitions to infinitely many cv repetitions
    evals_nonan = np.mean(np.mean(evaluations[eval_ok], -2), -2)
    evals_1 = np.mean(evaluations[eval_ok], -3)
    
    #if use_correction and n_cv > 1:
    matrix = evals_nonan
    matrix -= np.mean(matrix, 0, keepdims=True)    
    var_mean = np.einsum('ijk,ilk->kjl', matrix, matrix) \
        / (matrix.shape[0] - 1)    
    
    matrix_1 = evals_1
    matrix_1 -= np.mean(matrix_1, 0, keepdims=True)
    var_1 = np.einsum('ijmk,ilmk->kjl', matrix_1, matrix_1) \
        / (matrix_1.shape[0] - 1) / matrix_1.shape[2]    
    
    # this is the main formula for the correction:
    variances = (n_cv * var_mean - var_1) / (n_cv - 1)    
    
    # calcualte model variance and differential variance
    nc_included = False
    model_var, diff_var, noise_ceil_var = \
        extract_variances(variances, nc_included, n_rdm, n_pattern)
    
    return model_var, diff_var, variances, eval_ok

In [16]:
from rsatoolbox.util.matrix import pairwise_contrast
from rsatoolbox.util.rdm_utils import batch_to_matrices

def pairwise(evaluations, variances, dof):
    n_model = evaluations.shape[1]
    evaluations = np.nanmean(evaluations, 0)
    while evaluations.ndim > 1:
        evaluations = np.nanmean(evaluations, axis=-1)
    C = pairwise_contrast(np.arange(n_model))
    diffs = C @ evaluations
    t = diffs / np.sqrt(np.maximum(variances, np.finfo(float).eps))
    t = batch_to_matrices(np.array([t]))[0][0]
    p = 2 * (1 - stats.t.cdf(np.abs(t), dof))
    
    return diffs, t, p, evaluations


In [17]:
def bootstrap_pairwise(results):
    models = results[0].models
    n_model = results[0].n_model
    method = results[0].method
    cv_method = results[0].cv_method
    n_cv = 2
    fitter= results[0].fitter
    n_bootstraps=results[0].n_bootstraps

    evaluations = []
    variances = []
    dof = []
    n_rdm  = []
    n_pattern = []
    noise_ceil = []

    for result in results: # session 0 and session 1
        evaluations.append(result.evaluations)
        variances.append(result.variances)
        dof.append(result.dof)
        n_rdm.append(result.n_rdm)
        n_pattern.append(result.n_pattern)
        noise_ceil.append(result.noise_ceiling)


    eval_new = []
    #noise_ceil_new = []
    for i in range(n_model):
        temp = np.concatenate((evaluations[0][:,i:i+1,:,:],evaluations[1][:,i:i+1,:,:]),axis=1)
        #temp1 = np.concatenate((noise_ceil[0][:,i:i+1,:,:],evaluations[1][:,i:i+1,:,:]),axis=1)
        eval_new.append(temp)
        #noise_ceil.append()

    each_variance = []
    diff_variance = []
    nonan_iter = []

    for eval in eval_new:
        model_var, diff_var,variances, eval_ok = calc_model_var(eval,min(n_rdm),min(n_pattern))
        each_variance.append(model_var)
        diff_variance.append(diff_var)
        nonan_iter.append(sum(eval_ok))

    eval_diff = []
    t_stat = []
    p_val = []
    evals = []
    
    for i in range(len(eval_new)):
        diffs, t, p, eval = pairwise(eval_new[i], diff_variance[i], min(dof))
        eval_diff.append(diffs)
        t_stat.append(t)
        p_val.append(p)
        evals.append(eval)

    return eval_diff,t_stat,p_val,evals

In [189]:
all_tuples = (('V1', 'WT', 1),('V2', 'WT', 1), ('IPS0', 'WT', 1), ('SPL1', 'WT', 1),
              ('V1', 'Ecc', 1),('V2', 'Ecc', 1),('IPS0', 'Ecc', 1),('SPL1', 'Ecc', 1))

In [190]:
print(df1)

                                              results   roi  session     epoch
0   Results for running dual_bootstrap evaluation ...    M1        0     delay
0   Results for running dual_bootstrap evaluation ...    M1        1     delay
0   Results for running dual_bootstrap evaluation ...    V1        0     delay
0   Results for running dual_bootstrap evaluation ...    V1        1     delay
0   Results for running dual_bootstrap evaluation ...    V2        0     delay
..                                                ...   ...      ...       ...
0   Results for running dual_bootstrap evaluation ...  IPS4        1  stimulus
0   Results for running dual_bootstrap evaluation ...  IPS5        0  stimulus
0   Results for running dual_bootstrap evaluation ...  IPS5        1  stimulus
0   Results for running dual_bootstrap evaluation ...  SPL1        0  stimulus
0   Results for running dual_bootstrap evaluation ...  SPL1        1  stimulus

[90 rows x 4 columns]


In [191]:
data = []
model_names = [m.name for m in df1.iloc[0,0].models]
n_cv = 2

for one_tuple in all_tuples:
    roi = one_tuple[0]
    model = one_tuple[1]
    session = one_tuple[2]
    epochs = ['stimulus','delay']
    
    results = []
    for epoch in epochs:
        df = df1[(df1['roi']==roi) & (df1['session']==session) & (df1['epoch']==epoch)]
        #print(df)
        results.append(df.iloc[0,0])

    eval_diff,t_stat,p_val,evals = bootstrap_pairwise(results)

    idx = model_names.index(model)
    #df_epoch = pd eval_diff[idx][0],t_stat[idx][0,1],p_val[idx][0,1],evals[idx])
    data.append({
            'model': model,                
            'session': session,
            'roi': roi,
            'comparison': 'pairwise',
            'mean_stimulus': evals[idx][0],
            'mean_delay': evals[idx][1],
            'mean_diff':eval_diff[idx][0],
            't_stat': t_stat[idx][0,1],
            'p_value': p_val[idx][0,1],
        })


In [192]:
df_temp = pd.DataFrame(data)

In [193]:
print(df_temp)

  model  session   roi comparison  mean_stimulus  mean_delay  mean_diff  \
0    WT        1    V1   pairwise       0.230689    0.038062   0.192628   
1    WT        1    V2   pairwise       0.223397    0.106963   0.116435   
2    WT        1  IPS0   pairwise       0.090254    0.115978  -0.025724   
3    WT        1  SPL1   pairwise       0.051916    0.141067  -0.089150   
4   Ecc        1    V1   pairwise       0.217463   -0.005997   0.223460   
5   Ecc        1    V2   pairwise       0.246806    0.025533   0.221273   
6   Ecc        1  IPS0   pairwise       0.115812    0.142386  -0.026574   
7   Ecc        1  SPL1   pairwise       0.004675    0.157545  -0.152870   

     t_stat   p_value  
0  3.935278  0.017027  
1  1.626054  0.179265  
2 -0.286445  0.788761  
3 -1.009616  0.369792  
4  2.534619  0.064348  
5  2.788844  0.049370  
6 -0.717428  0.512775  
7 -1.358002  0.246014  


In [194]:
df_temp1 = df_temp[df_temp['model']=='WT']
df_temp2 = df_temp[df_temp['model']=='Ecc']
df_temp1 = multi_corr(df_temp1)
df_temp2 = multi_corr(df_temp2)

df_temp_new = pd.concat((df_temp1,df_temp2),axis=0)
print(df_temp_new)

  model  session   roi comparison  mean_stimulus  mean_delay  mean_diff  \
0    WT        1    V1   pairwise       0.230689    0.038062   0.192628   
1    WT        1    V2   pairwise       0.223397    0.106963   0.116435   
2    WT        1  IPS0   pairwise       0.090254    0.115978  -0.025724   
3    WT        1  SPL1   pairwise       0.051916    0.141067  -0.089150   
4   Ecc        1    V1   pairwise       0.217463   -0.005997   0.223460   
5   Ecc        1    V2   pairwise       0.246806    0.025533   0.221273   
6   Ecc        1  IPS0   pairwise       0.115812    0.142386  -0.026574   
7   Ecc        1  SPL1   pairwise       0.004675    0.157545  -0.152870   

     t_stat   p_value  p_value_corr  
0  3.935278  0.017027      0.068109  
1  1.626054  0.179265      0.358531  
2 -0.286445  0.788761      0.788761  
3 -1.009616  0.369792      0.493057  
4  2.534619  0.064348      0.128696  
5  2.788844  0.049370      0.128696  
6 -0.717428  0.512775      0.512775  
7 -1.358002  0.24601

In [195]:

# multiple correction across all
df_temp = multi_corr(df_temp)
print(df_temp)


  model  session   roi comparison  mean_stimulus  mean_delay  mean_diff  \
0    WT        1    V1   pairwise       0.230689    0.038062   0.192628   
1    WT        1    V2   pairwise       0.223397    0.106963   0.116435   
2    WT        1  IPS0   pairwise       0.090254    0.115978  -0.025724   
3    WT        1  SPL1   pairwise       0.051916    0.141067  -0.089150   
4   Ecc        1    V1   pairwise       0.217463   -0.005997   0.223460   
5   Ecc        1    V2   pairwise       0.246806    0.025533   0.221273   
6   Ecc        1  IPS0   pairwise       0.115812    0.142386  -0.026574   
7   Ecc        1  SPL1   pairwise       0.004675    0.157545  -0.152870   

     t_stat   p_value  p_value_corr  
0  3.935278  0.017027      0.136218  
1  1.626054  0.179265      0.358531  
2 -0.286445  0.788761      0.788761  
3 -1.009616  0.369792      0.493057  
4  2.534619  0.064348      0.171594  
5  2.788844  0.049370      0.171594  
6 -0.717428  0.512775      0.586029  
7 -1.358002  0.24601

In [196]:
df_temp.to_csv('tbl_bootstrap_RDM_accuracy_pairwise_sp_ecc_sess2.csv')

In [197]:
df_temp_sess2 = df_temp

In [198]:
all_tuples = (('V1', 'WT', 0),('V2', 'WT', 0), ('IPS0', 'WT', 0), ('SPL1', 'WT', 0),
              ('V1', 'Ecc', 0),('V2', 'Ecc', 0),('IPS0', 'Ecc', 0),('SPL1', 'Ecc', 0))

data = []
model_names = [m.name for m in df1.iloc[0,0].models]
n_cv = 2

for one_tuple in all_tuples:
    roi = one_tuple[0]
    model = one_tuple[1]
    session = one_tuple[2]
    epochs = ['stimulus','delay']
    
    results = []
    for epoch in epochs:
        df = df1[(df1['roi']==roi) & (df1['session']==session) & (df1['epoch']==epoch)]
        #print(df)
        results.append(df.iloc[0,0])

    eval_diff,t_stat,p_val,evals = bootstrap_pairwise(results)

    idx = model_names.index(model)
    #df_epoch = pd eval_diff[idx][0],t_stat[idx][0,1],p_val[idx][0,1],evals[idx])
    data.append({
            'model': model,                
            'session': session,
            'roi': roi,
            'comparison': 'pairwise',
            'mean_stimulus': evals[idx][0],
            'mean_delay': evals[idx][1],
            'mean_diff':eval_diff[idx][0],
            't_stat': t_stat[idx][0,1],
            'p_value': p_val[idx][0,1],
        })
    
df_temp = pd.DataFrame(data)

# multiple correction across all
df_temp = multi_corr(df_temp)

print(df_temp)


  model  session   roi comparison  mean_stimulus  mean_delay  mean_diff  \
0    WT        0    V1   pairwise       0.178516    0.096234   0.082282   
1    WT        0    V2   pairwise       0.165382    0.044339   0.121043   
2    WT        0  IPS0   pairwise       0.022315    0.128713  -0.106397   
3    WT        0  SPL1   pairwise      -0.053202    0.038891  -0.092093   
4   Ecc        0    V1   pairwise       0.204376   -0.019025   0.223401   
5   Ecc        0    V2   pairwise       0.233801   -0.084086   0.317887   
6   Ecc        0  IPS0   pairwise       0.136649    0.050241   0.086408   
7   Ecc        0  SPL1   pairwise      -0.018548    0.088198  -0.106746   

     t_stat   p_value  p_value_corr  
0  1.418403  0.215281      0.328132  
1  2.268063  0.072615      0.193639  
2 -1.683535  0.153092      0.306184  
3 -1.083288  0.328132      0.328132  
4  3.312090  0.021192      0.084767  
5  3.761569  0.013136      0.084767  
6  1.188780  0.287903      0.328132  
7 -1.224467  0.27531

In [188]:
df_temp.to_csv('tbl_bootstrap_RDM_accuracy_pairwise_sp_ecc_sess1.csv')

In [199]:
df_temp_sess1 = df_temp

In [201]:
df_temp_sess_all = pd.concat((df_temp_sess1,df_temp_sess2),axis=0)

In [202]:
# multiple correction across all
df_temp_sess_all = multi_corr(df_temp_sess_all)

In [203]:
df_temp_sess_all.to_csv('tbl_bootstrap_RDM_accuracy_pairwise_sp_ecc_sess_all.csv')

# pairwise across models

In [204]:
# combinations
visual_comb1 = (('V1',0,'stimulus',['WT','Ecc','Ecc-WT'],['WT vs Ecc-WT','WT vs Ecc','Ecc vs Ecc-WT']),
               ('V1',0,'stimulus',['LR','WT','LR-WT'],['LR vs LR-WT','LR vs WT','WT vs LR-WT']),
               ('V1',0,'stimulus',['LR','Ecc','Ecc-LR'],['LR vs Ecc-LR','LR vs Ecc','Ecc vs Ecc-LR']),
               ('V2',0,'stimulus',['WT','Ecc','Ecc-WT'],['WT vs Ecc-WT','WT vs Ecc','Ecc vs Ecc-WT']),
               ('V2',0,'stimulus',['LR','WT','LR-WT'],['LR vs LR-WT','LR vs WT','WT vs LR-WT']),
               ('V2',0,'stimulus',['LR','Ecc','Ecc-LR'],['LR vs Ecc-LR','LR vs Ecc','Ecc vs Ecc-LR']))

visual_comb2 = (('V1',1,'stimulus',['WT','Ecc','Ecc-WT'],['WT vs Ecc-WT','WT vs Ecc','Ecc vs Ecc-WT']),
               ('V1',1,'stimulus',['LR','WT','LR-WT'],['LR vs LR-WT','LR vs WT','WT vs LR-WT']),
               ('V1',1,'stimulus',['LR','Ecc','Ecc-LR'],['LR vs Ecc-LR','LR vs Ecc','Ecc vs Ecc-LR']),
               ('V2',1,'stimulus',['WT','Ecc','Ecc-WT'],['WT vs Ecc-WT','WT vs Ecc','Ecc vs Ecc-WT']),
               ('V2',1,'stimulus',['LR','WT','LR-WT'],['LR vs LR-WT','LR vs WT','WT vs LR-WT']),
               ('V2',1,'stimulus',['LR','Ecc','Ecc-LR'],['LR vs Ecc-LR','LR vs Ecc','Ecc vs Ecc-LR']))

parietal_comb1 = (('IPS2',0,'delay',['WT','Ecc','Ecc-WT'],['WT vs Ecc-WT','WT vs Ecc','Ecc vs Ecc-WT']),
               ('IPS2',0,'delay',['LR','WT','LR-WT'],['LR vs LR-WT','LR vs WT','WT vs LR-WT']))

parietal_comb2 = (('IPS0',1,'delay',['WT','Ecc','Ecc-WT'],['WT vs Ecc-WT','WT vs Ecc','Ecc vs Ecc-WT']),
               ('IPS1',1,'delay',['LR','WT','LR-WT'],['LR vs LR-WT','LR vs WT','WT vs LR-WT']),
               ('SPL1',1,'delay',['WT','Ecc','Ecc-WT'],['WT vs Ecc-WT','WT vs Ecc','Ecc vs Ecc-WT']),
               ('SPL1',1,'delay',['LR','WT','LR-WT'],['LR vs LR-WT','LR vs WT','WT vs LR-WT']))

In [219]:
all_comb = visual_comb1+visual_comb2+parietal_comb1+parietal_comb2
print(all_comb)

df_comb = pd.DataFrame()

for comb in all_comb:
    df = df_results[(df_results['roi'].isin([comb[0]])) 
                    & (df_results['session'].isin([comb[1]])) 
                    & (df_results['epoch'].isin([comb[2]]))
                    & (df_results['comparison'].isin(['pairwise']))
                    & (df_results['model'].isin(comb[4]))]
    
    df = multi_corr(df)
    
    df_comb = pd.concat((df_comb,df),axis=0)
    

(('V1', 0, 'stimulus', ['WT', 'Ecc', 'Ecc-WT'], ['WT vs Ecc-WT', 'WT vs Ecc', 'Ecc vs Ecc-WT']), ('V1', 0, 'stimulus', ['LR', 'WT', 'LR-WT'], ['LR vs LR-WT', 'LR vs WT', 'WT vs LR-WT']), ('V1', 0, 'stimulus', ['LR', 'Ecc', 'Ecc-LR'], ['LR vs Ecc-LR', 'LR vs Ecc', 'Ecc vs Ecc-LR']), ('V2', 0, 'stimulus', ['WT', 'Ecc', 'Ecc-WT'], ['WT vs Ecc-WT', 'WT vs Ecc', 'Ecc vs Ecc-WT']), ('V2', 0, 'stimulus', ['LR', 'WT', 'LR-WT'], ['LR vs LR-WT', 'LR vs WT', 'WT vs LR-WT']), ('V2', 0, 'stimulus', ['LR', 'Ecc', 'Ecc-LR'], ['LR vs Ecc-LR', 'LR vs Ecc', 'Ecc vs Ecc-LR']), ('V1', 1, 'stimulus', ['WT', 'Ecc', 'Ecc-WT'], ['WT vs Ecc-WT', 'WT vs Ecc', 'Ecc vs Ecc-WT']), ('V1', 1, 'stimulus', ['LR', 'WT', 'LR-WT'], ['LR vs LR-WT', 'LR vs WT', 'WT vs LR-WT']), ('V1', 1, 'stimulus', ['LR', 'Ecc', 'Ecc-LR'], ['LR vs Ecc-LR', 'LR vs Ecc', 'Ecc vs Ecc-LR']), ('V2', 1, 'stimulus', ['WT', 'Ecc', 'Ecc-WT'], ['WT vs Ecc-WT', 'WT vs Ecc', 'Ecc vs Ecc-WT']), ('V2', 1, 'stimulus', ['LR', 'WT', 'LR-WT'], ['LR vs LR-W

In [220]:
#df_comb = multi_corr(df_comb)

In [221]:
df_comb['mean1'] = df_comb['mean'].apply(lambda x: x[0])
df_comb['mean2'] = df_comb['mean'].apply(lambda x: x[1])

In [222]:
df_temp = df_comb[df_comb['p_value']<0.05]

In [223]:
print(df_temp)

             model  session     epoch   roi comparison  \
253       LR vs WT        0  stimulus    V1   pairwise   
254      LR vs Ecc        0  stimulus    V1   pairwise   
256   LR vs Ecc-LR        0  stimulus    V1   pairwise   
516   WT vs Ecc-WT        0  stimulus    V2   pairwise   
505       LR vs WT        0  stimulus    V2   pairwise   
507    LR vs LR-WT        0  stimulus    V2   pairwise   
506      LR vs Ecc        0  stimulus    V2   pairwise   
508   LR vs Ecc-LR        0  stimulus    V2   pairwise   
316       LR vs WT        1  stimulus    V1   pairwise   
318    LR vs LR-WT        1  stimulus    V1   pairwise   
319   LR vs Ecc-LR        1  stimulus    V1   pairwise   
568       LR vs WT        1  stimulus    V2   pairwise   
570    LR vs LR-WT        1  stimulus    V2   pairwise   
569      LR vs Ecc        1  stimulus    V2   pairwise   
571   LR vs Ecc-LR        1  stimulus    V2   pairwise   
2460   LR vs LR-WT        1     delay  IPS1   pairwise   

             

In [218]:
df_temp.to_csv('tbl_bootstrap_RDM_accuracy_pairwise_model.csv')

In [236]:
all_comb = (('V1',1,'stimulus',['LR','WT','Ecc','LR-WT','Ecc-LR','Ecc-WT'],
             ['WT vs Ecc-WT','WT vs Ecc','Ecc vs Ecc-WT',
              'LR vs LR-WT','LR vs WT','WT vs LR-WT',
              'LR vs Ecc-LR','LR vs Ecc','Ecc vs Ecc-LR']),
('V2',1,'stimulus',['LR','WT','Ecc','LR-WT','Ecc-LR','Ecc-WT'],
             ['WT vs Ecc-WT','WT vs Ecc','Ecc vs Ecc-WT',
              'LR vs LR-WT','LR vs WT','WT vs LR-WT',
              'LR vs Ecc-LR','LR vs Ecc','Ecc vs Ecc-LR']),               
('IPS0',1,'delay',['LR','WT','Ecc','LR-WT','Ecc-LR','Ecc-WT'],
             ['WT vs Ecc-WT','WT vs Ecc','Ecc vs Ecc-WT',
              'LR vs LR-WT','LR vs WT','WT vs LR-WT',
              'LR vs Ecc-LR','LR vs Ecc','Ecc vs Ecc-LR']),              
('SPL1',1,'delay',['LR','WT','Ecc','LR-WT','Ecc-LR','Ecc-WT'],
             ['WT vs Ecc-WT','WT vs Ecc','Ecc vs Ecc-WT',
              'LR vs LR-WT','LR vs WT','WT vs LR-WT',
              'LR vs Ecc-LR','LR vs Ecc','Ecc vs Ecc-LR']))


In [237]:

df_comb = pd.DataFrame()

for comb in all_comb:

    df = df_results[(df_results['roi'].isin([comb[0]])) 
                 #   & (df_results['session'].isin([comb[1]])) 
                    & (df_results['epoch'].isin([comb[2]]))
                    & (df_results['comparison'].isin(['pairwise']))
                    & (df_results['model'].isin(comb[4]))]
    
    df = multi_corr(df)
    
    df_comb = pd.concat((df_comb,df),axis=0)

In [238]:
df_comb['mean1'] = df_comb['mean'].apply(lambda x: x[0])
df_comb['mean2'] = df_comb['mean'].apply(lambda x: x[1])

In [239]:
df_temp = df_comb[df_comb['p_value']<0.05]
print(df_temp)

            model  session     epoch roi comparison  \
253      LR vs WT        0  stimulus  V1   pairwise   
254     LR vs Ecc        0  stimulus  V1   pairwise   
256  LR vs Ecc-LR        0  stimulus  V1   pairwise   
316      LR vs WT        1  stimulus  V1   pairwise   
318   LR vs LR-WT        1  stimulus  V1   pairwise   
319  LR vs Ecc-LR        1  stimulus  V1   pairwise   
505      LR vs WT        0  stimulus  V2   pairwise   
506     LR vs Ecc        0  stimulus  V2   pairwise   
507   LR vs LR-WT        0  stimulus  V2   pairwise   
508  LR vs Ecc-LR        0  stimulus  V2   pairwise   
516  WT vs Ecc-WT        0  stimulus  V2   pairwise   
568      LR vs WT        1  stimulus  V2   pairwise   
569     LR vs Ecc        1  stimulus  V2   pairwise   
570   LR vs LR-WT        1  stimulus  V2   pairwise   
571  LR vs Ecc-LR        1  stimulus  V2   pairwise   

                                            mean  mean_diff   p_value  \
253  [0.016844436127424384, 0.1785161645188190

In [240]:
df_temp.to_csv('tbl_bootstrap_RDM_accuracy_pairwise_model_seven.csv')

# pairwise across scanning sessions

In [18]:
df_temp = df_results[(df_results['comparison']=='against_zero') & 
          (df_results['roi'].isin(['V1','V2'])) &
          (df_results['epoch']=='stimulus') & 
          (df_results['p_value']<=0.05) & 
          (df_results['model']!='All') #&
          ] #(df_results['session']==0)
print(df_temp)

      model  session     epoch roi    comparison      mean  mean_diff  \
302      WT        0  stimulus  V1  against_zero  0.178516        NaN   
303     Ecc        0  stimulus  V1  against_zero  0.204376        NaN   
304   LR-WT        0  stimulus  V1  against_zero  0.187566        NaN   
305  Ecc-LR        0  stimulus  V1  against_zero  0.212496        NaN   
306  Ecc-WT        0  stimulus  V1  against_zero  0.243389        NaN   
365      WT        1  stimulus  V1  against_zero  0.230689        NaN   
366     Ecc        1  stimulus  V1  against_zero  0.217463        NaN   
367   LR-WT        1  stimulus  V1  against_zero  0.259624        NaN   
368  Ecc-LR        1  stimulus  V1  against_zero  0.258181        NaN   
369  Ecc-WT        1  stimulus  V1  against_zero  0.317809        NaN   
554      WT        0  stimulus  V2  against_zero  0.165382        NaN   
555     Ecc        0  stimulus  V2  against_zero  0.233801        NaN   
556   LR-WT        0  stimulus  V2  against_zero  0

In [19]:
df_temp = df_results[(df_results['comparison']=='against_zero') & 
          (df_results['roi'].isin(['IPS0','IPS1','IPS2','SPL1'])) &
          (df_results['epoch']=='delay') & 
          (df_results['p_value']<=0.05) & 
          (df_results['model']!='All')] # &           (df_results['session']==0)
print(df_temp)

       model  session  epoch   roi    comparison      mean  mean_diff  \
2255      WT        1  delay  IPS0  against_zero  0.115978        NaN   
2256     Ecc        1  delay  IPS0  against_zero  0.142386        NaN   
2509   LR-WT        1  delay  IPS1  against_zero  0.152516        NaN   
2696      WT        0  delay  IPS2  against_zero  0.101306        NaN   
3767      WT        1  delay  SPL1  against_zero  0.141067        NaN   
3768     Ecc        1  delay  SPL1  against_zero  0.157545        NaN   
3769   LR-WT        1  delay  SPL1  against_zero  0.106844        NaN   
3771  Ecc-WT        1  delay  SPL1  against_zero  0.239361        NaN   

       p_value       sem  
2255  0.021340   0.03954  
2256  0.035178  0.058089  
2509  0.019573  0.050495  
2696  0.041355  0.046802  
3767  0.028130  0.052988  
3768  0.041166  0.068299  
3769  0.024812   0.03838  
3771  0.005236  0.052688  


In [20]:
# all_comb0 = (('v1','stimulus','WT'),('v1','stimulus','Ecc'),
#             ('v1','stimulus','LR-WT'),('v1','stimulus','Ecc-LR'),('v1','stimulus','Ecc-WT'),
#             ('v2','stimulus','LR'),('v2','stimulus','WT'),('v2','stimulus','Ecc'),
#             ('v1','stimulus','LR-WT'),('v1','stimulus','Ecc-LR'),('v1','stimulus','Ecc-WT'),
#             ('ips0','delay','WT'),('v1','delay','Ecc'),
#             ('ips1','delay','LR-WT'),
#             ('ips2','delay','WT'),
#             ('spl1','delay','WT'),('spl1','delay','Ecc'),('spl1','delay','LR-WT'),('spl1','delay','Ecc-WT'))

In [23]:
all_comb = (('V1','stimulus','WT'),('V1','stimulus','Ecc'),
            ('V2','stimulus','WT'),('V2','stimulus','Ecc'),
            ('IPS0','delay','WT'),('IPS0','delay','Ecc'),
            ('SPL1','delay','WT'),('SPL1','delay','Ecc'))

In [30]:
data = []
model_names = [m.name for m in df1.iloc[0,0].models]

for comb in all_comb:
    sessions = [0,1]
    epoch = comb[1]
    roi = comb[0]
    model = comb[2]
    n_cv = 2
    
    results = []
    for session in sessions:
        df = df1[(df1['roi']==roi) & (df1['session']==session) & (df1['epoch']==epoch)]
        results.append(df.iloc[0,0])

    eval_diff,t_stat,p_val,evals = bootstrap_pairwise(results)

    idx = model_names.index(model)
    data.append({'roi': roi,
                 'epoch': epoch,
                 'model': model,
                 'eval_sess1': evals[idx][0],
                 'eval_sess2': evals[idx][1],
                 'eval_diff': eval_diff[idx][0],
                 't_stat': t_stat[idx][0,1],
                 'p_value': p_val[idx][0,1]})

In [31]:
df_temp = pd.DataFrame(data)
print(df_temp)

    roi     epoch model  eval_sess1  eval_sess2  eval_diff    t_stat   p_value
0    V1  stimulus    WT    0.178516    0.230689  -0.052173 -1.217248  0.290411
1    V1  stimulus   Ecc    0.204376    0.217463  -0.013087 -0.224306  0.833511
2    V2  stimulus    WT    0.165382    0.223397  -0.058016 -1.222549  0.288608
3    V2  stimulus   Ecc    0.233801    0.246806  -0.013005 -0.217638  0.838363
4  IPS0     delay    WT    0.128713    0.115978   0.012734  0.158102  0.882037
5  IPS0     delay   Ecc    0.050241    0.142386  -0.092145 -1.619963  0.180555
6  SPL1     delay    WT    0.038891    0.141067  -0.102175 -1.502372  0.207418
7  SPL1     delay   Ecc    0.088198    0.157545  -0.069347 -0.606872  0.576696


In [32]:
df_temp = multi_corr(df_temp)

In [33]:
df_temp.to_csv('tbl_bootstrap_RDM_accuracy_pairwise_sess.csv')